In [5]:
import cv2
import numpy as np
import os

In [2]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
while True:
    ret,frame=cap.read()
    if(ret==False):
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
  
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.imshow("video frame",frame)
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [8]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
skip=0
face_data=[]
dataset_path='./cvdata/'
filename=input()
while True:
    ret,frame=cap.read()
    if(ret==False):
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    for face in faces[-1:]:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        skip+=1
        if(skip%10==0):
            face_data.append(face_section)
            #print(len(face_data))
            
    cv2.imshow("video frame",frame)
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
        break

face_data=np.asarray(face_data)
#print(face_data.shape)
face_data=face_data.reshape((face_data.shape[0],-1))
np.save(dataset_path+filename+'.npy',face_data)
cap.release()
cv2.destroyAllWindows()
       
    
    

ayush


In [9]:
def distance(x1,x2):
    return np.sqrt(sum((x1-x2)**2))
def knn(train,test,k=5):
    dist=[]
    for i in range(train.shape[0]):
        ix=train[i,:-1]
        iy=train[i,-1]
        d=distance(test,ix)
        dist.append([d,iy])
    dk=sorted(dist)
    dk=dk[:k]
    labels=np.array(dk)[:,-1]
    output=np.unique(labels,return_counts=True)
    index=np.argmax(output[1])
    return output[0][index]
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
skip=0
dataset_path='./cvdata/'
face_data=[]
labels=[]
class_id=0 #labels for the given file
names={}  #mapping between id and names
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]
        data_items=np.load(dataset_path+fx)
        #print(data_items)
        #print(data_items.shape)
        face_data.append(data_items)
        #print(face_data)
        #print(face_data.shape)
        target=class_id*np.ones((data_items.shape[0]))
        class_id+=1
        labels.append(target)
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
#print(face_dataset.shape)
#print(face_dataset)
#print(face_labels.shape)
#print(face_labels)
trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset)
print(trainset.shape)
while True:
    ret,frame=cap.read()
    if(ret == False):
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h=face
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        out=knn(trainset,face_section.flatten())
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("Faces",frame)
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    
        
        
    


[[ 57.  60.  48. ...  53.  52.   0.]
 [ 57.  58.  53. ...  55.  58.   0.]
 [ 99.  91.  79. ... 168. 161.   0.]
 ...
 [165. 166. 162. ... 142. 141.   2.]
 [135. 138. 127. ... 145. 146.   2.]
 [ 73.  74.  67. ... 145. 144.   2.]]
(27, 30001)
